In [3]:
import random
import os

import numpy as np
import pandas as pd

from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import missingno

# device 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print (device)

seed = 42 # seed 값 설정
random.seed(seed) # 파이썬 난수 생성기
os.environ['PYTHONHASHSEED'] = str(seed) # 해시 시크릿값 고정
np.random.seed(seed) # 넘파이 난수 생성기

torch.manual_seed(seed) # 파이토치 CPU 난수 생성기
torch.backends.cudnn.deterministic = True # 확정적 연산 사용 설정
torch.backends.cudnn.benchmark = False   # 벤치마크 기능 사용 해제
torch.backends.cudnn.enabled = False        # cudnn 기능 사용 해제

if device == 'cuda':
    torch.cuda.manual_seed(seed) # 파이토치 GPU 난수 생성기
    torch.cuda.manual_seed_all(seed) # 파이토치 멀티 GPU 난수 생성기

cpu


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [83]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AIDrug_Competition/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AIDrug_Competition/data/test.csv')

In [84]:
train

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.259,117.37
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.172,73.47
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.585,62.45
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.475,92.60
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.337,42.43
...,...,...,...,...,...,...,...,...,...,...,...
3493,TRAIN_3493,Cn1nc(CNC(=O)Cn2nc(C(F)(F)F)c3c2CCC3)c(Cl)c1Cl,1.556,3.079,3.409,396.195,3,1,5,3.409,64.74
3494,TRAIN_3494,CCn1[nH]cc/c1=N\C(=O)c1nn(-c2ccccc2)c(=O)c2ccc...,35.560,47.630,1.912,359.381,4,1,3,1.844,77.37
3495,TRAIN_3495,CCOC(=O)CCCc1nc2cc(N)ccc2n1C,56.150,1.790,1.941,261.320,3,1,6,2.124,70.14
3496,TRAIN_3496,Nc1cc(C(=O)OCCC2CCOC2=O)cnc1Cl,0.030,2.770,0.989,284.696,5,1,5,0.989,91.51


In [85]:
!pip install rdkit-pypi

In [86]:
import pandas as pd
import numpy as np
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors

def calculate_metabolic_stability_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    logP = Descriptors.MolLogP(mol)
    # 화합물의 친유성을 측정한 것으로 지질 또는 비극성 환경에서의 용해도를 나타냅니다. 생물학적 막을 통과하는 화합물의 능력을 반영합니다.
    apka = Descriptors.MolWt(mol)
    # 화합물의 산 해리 상수의 추정치로 다양한 pH 조건에서 이온화 거동에 대한 정보를 제공합니다.
    num_rotatable_bonds = Descriptors.NumRotatableBonds(mol)
    # 화합물에서 회전 가능한 결합의 수입니다. 이것은 화합물의 유연성과 효소 또는 다른 분자와의 잠재적인 상호 작용에 대한 통찰력을 제공할 수 있습니다.
    num_heteroatoms = Descriptors.NumHeteroatoms(mol)
    # 분자 내 헤테로원자(탄소 및 수소 이외의 원자) 수. 이는 화합물의 반응성과 대사 안정성에 영향을 줄 수 있습니다.
    num_hydrogen_acceptors = Descriptors.NumHAcceptors(mol)
    # 분자 내 수소 결합 수용체의 수. 이들은 결합 및 반응성에 영향을 미치는 다른 분자의 수소 결합 기증자와 상호 작용할 수 있는 사이트입니다.
    num_hydrogen_donors = Descriptors.NumHDonors(mol)
    # 분자 내 수소 결합 기증자의 수입니다. 이들은 수소 결합 상호작용에서 수소 원자를 제공할 수 있는 사이트입니다.
    # morgan_fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
    morgan_fingerprint = AllChem.GetHashedMorganFingerprint(mol, 6, nBits=4096)
    # 분자 하위 구조의 이진 벡터 표현입니다. 이 열에는 화합물과 효소의 상호 작용 및 대사 안정성에 영향을 줄 수 있는 구조적 특징을 포착하는 이진 지문이 포함되어 있습니다.
    morgan_array = np.zeros((1,), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(morgan_fingerprint, morgan_array)

    return logP, apka, num_rotatable_bonds, num_heteroatoms, num_hydrogen_acceptors, num_hydrogen_donors, morgan_array

train[[
    'logP', 'apka', 'num_rotatable_bonds', 'num_heteroatoms',
    'num_hydrogen_acceptors', 'num_hydrogen_donors', 'morgan_fingerprint'
]] = train['SMILES'].apply(calculate_metabolic_stability_descriptors).apply(pd.Series)

test[[
    'logP', 'apka', 'num_rotatable_bonds', 'num_heteroatoms',
    'num_hydrogen_acceptors', 'num_hydrogen_donors', 'morgan_fingerprint'
]] = test['SMILES'].apply(calculate_metabolic_stability_descriptors).apply(pd.Series)

test.head()


,id,SMILES,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,logP,apka,num_rotatable_bonds,num_heteroatoms,num_hydrogen_acceptors,num_hydrogen_donors,morgan_fingerprint
0,TEST_000,CC(C)Nc1ccnc(N2CCN(Cc3cccs3)C(CCO)C2)n1,2.641,361.505,4,2,7,2.635,92.76,2.43160,361.515,7,7,7,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,TEST_001,COc1cc(=O)n(-c2ccccc2)cc1C(=O)N1CCC2(CC1)OCCO2,0.585,370.399,5,0,3,0.585,68.31,1.82520,370.405,3,7,6,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,TEST_002,Cc1cccc(NC(=N)/N=c2\nc(O)c(Cc3ccccc3)c(C)[nH]2)c1,4.276,347.414,4,4,5,4.290,92.86,3.27051,347.422,3,6,3,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,TEST_003,O=C(c1nc2ncccn2n1)N1CCCn2cc(-c3ccccc3)nc21,1.795,345.358,5,0,2,1.795,81.21,2.03830,345.366,2,8,7,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,TEST_004,CCN1CCN(C(=O)c2cc3c(=O)n4cc(C)ccc4nc3n2C)CC1,1.219,353.418,4,0,2,0.169,61.15,1.27232,353.426,2,7,6,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [87]:
train['morgan_fingerprint'].info() # (3498, 2048)

<class 'pandas.core.series.Series'>
RangeIndex: 3498 entries, 0 to 3497
Series name: morgan_fingerprint
Non-Null Count  Dtype 
--------------  ----- 
3498 non-null   object
dtypes: object(1)
memory usage: 27.5+ KB


In [88]:
train = train.fillna(train['AlogP'].median())

In [89]:
class CustomDataset(Dataset):
    def __init__(self, df, target, transform, is_test=False):
        self.df = df
        self.target = target # HLM or MLM
        self.is_test = is_test # train,valid / test

        self.feature_select = transform
        if not self.is_test:
            self.fp = self.feature_select.fit_transform(np.stack(df['morgan_fingerprint']))
            # self.fp = np.stack(df['morgan_fingerprint'])
        else: # valid or test
            self.fp = self.feature_select.transform(np.stack(df['morgan_fingerprint']))
            # self.fp = np.stack(df['morgan_fingerprint'])

    def __getitem__(self, index):
        fp = self.fp[index]
        if not self.is_test: # test가 아닌 경우(label 존재)
            label = self.df[self.target][index]
            return torch.tensor(fp).to(device).float(), torch.tensor(label).to(device).float().unsqueeze(dim=-1) # feature, label

        else: # test인 경우
            return torch.tensor(fp).to(device).float() # feature

    def __len__(self):
        return len(self.df)

In [90]:
transform = VarianceThreshold(threshold=0.05)

train_MLM = CustomDataset(df=train, target='MLM', transform=transform, is_test=False)
train_HLM = CustomDataset(df=train, target='HLM', transform=transform, is_test=False)

input_size = train_MLM.fp.shape[1]
input_size

251

In [91]:
train_HLM.fp.shape

(3498, 251)

In [92]:
# train,valid split
train_MLM_dataset, valid_MLM_dataset = train_test_split(train_MLM, test_size=0.2, random_state=42)
train_HLM_dataset, valid_HLM_dataset = train_test_split(train_HLM, test_size=0.2, random_state=42)

In [93]:
torch.tensor(train_MLM.fp[1]).shape, torch.tensor(train['MLM'][1]).float().unsqueeze(dim=-1).shape

(torch.Size([251]), torch.Size([1]))

In [94]:
# Hyperparameter
CFG = {'BATCH_SIZE': 256,
       'EPOCHS': 5000,
       'INPUT_SIZE': input_size,
       'HIDDEN_SIZE': 1024,
       'OUTPUT_SIZE': 1,
       'DROPOUT_RATE': 0.8,
       'LEARNING_RATE': 0.001}

In [95]:
train_MLM_loader = DataLoader(dataset=train_MLM_dataset,
                              batch_size=CFG['BATCH_SIZE'],
                              shuffle=True)

valid_MLM_loader = DataLoader(dataset=valid_MLM_dataset,
                              batch_size=CFG['BATCH_SIZE'],
                              shuffle=False)


train_HLM_loader = DataLoader(dataset=train_HLM_dataset,
                              batch_size=CFG['BATCH_SIZE'],
                              shuffle=True)

valid_HLM_loader = DataLoader(dataset=valid_HLM_dataset,
                              batch_size=CFG['BATCH_SIZE'],
                              shuffle=False)

In [96]:
X_train, y_train = next(iter(train_MLM_loader))
print (X_train.shape, y_train.shape)

torch.Size([256, 251]) torch.Size([256, 1])


In [97]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_rate, out_size):
        super(Net, self).__init__()

        self.fc_layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.LeakyReLU(),
            nn.Dropout(dropout_rate),

            nn.Linear(hidden_size, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.LeakyReLU(),
            nn.Dropout(dropout_rate),

            nn.Linear(hidden_size, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.LeakyReLU(),
            nn.Dropout(dropout_rate)
        )

        self.fc_out = nn.Linear(hidden_size, out_size)

    def forward(self, x):
        out = self.fc_layers(x)
        out = self.fc_out(out)
        return out


In [98]:
model_MLM = Net(CFG['INPUT_SIZE'],CFG['HIDDEN_SIZE'],CFG['DROPOUT_RATE'],CFG['OUTPUT_SIZE'])
model_HLM = Net(CFG['INPUT_SIZE'],CFG['HIDDEN_SIZE'],CFG['DROPOUT_RATE'],CFG['OUTPUT_SIZE'])

In [99]:
import torch.nn as nn

class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        self.mse = nn.MSELoss()  # 기존의 MSELoss 함수 사용

    def forward(self, output, target):
        mse_loss = self.mse(output, target)  # 기존의 MSELoss를 계산
        rmse_loss = torch.sqrt(mse_loss)  # MSE에 제곱근 씌워 RMSE 계산
        return rmse_loss

criterion = RMSELoss()
optimizer_MLM = torch.optim.Adam(model_MLM.parameters(), lr=CFG['LEARNING_RATE'])
optimizer_HLM = torch.optim.Adam(model_HLM.parameters(), lr=CFG['LEARNING_RATE'])


In [100]:
def train(train_loader, valid_loader, model, criterion, optimizer, epochs, patience=10):
    best_valid_loss = float('inf')
    no_improvement_count = 0

    for epoch in range(epochs):
        model.train()  # 모델을 훈련 모드로 설정
        running_loss = 0
        for inputs, targets in train_loader:
            optimizer.zero_grad()

            output = model(inputs)
            loss = criterion(output, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        model.eval()  # 모델을 검증 모드로 설정
        valid_loss = 0
        with torch.no_grad():
          for inputs, targets in valid_loader:
            output = model(inputs)
            loss = criterion(output, targets)
            valid_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        avg_valid_loss = valid_loss / len(valid_loader)
        print(f'Epoch: {epoch}/{epochs}, Train Loss: {avg_train_loss}, Valid Loss: {avg_valid_loss}')

        if avg_valid_loss < best_valid_loss:
          best_valid_loss = avg_valid_loss
          no_improvement_count = 0
          best_model_state = model.state_dict()
        else:
          no_improvement_count += 1
          if no_improvement_count >= patience:
            print(f'얼리 스토핑: {patience} 에포크 동안 검증 손실이 향상되지 않음. 에포크 {epoch}에서 훈련 중단.')
            break

    # 최적의 모델 상태 불러오기
    model.load_state_dict(best_model_state)

    return model

In [101]:
print("Training Start: MLM")
model_MLM = train(train_MLM_loader, valid_MLM_loader, model_MLM, criterion, optimizer_MLM, epochs=CFG['EPOCHS'])

print("Training Start: HLM")
model_HLM = train(train_HLM_loader, valid_HLM_loader, model_HLM, criterion, optimizer_HLM, epochs=CFG['EPOCHS'])

Training Start: MLM
Epoch: 0/5000, Train Loss: 45.24780134721236, Valid Loss: 42.624427795410156
Epoch: 1/5000, Train Loss: 40.35728073120117, Valid Loss: 39.94778696695963
Epoch: 2/5000, Train Loss: 38.01994254372337, Valid Loss: 38.095776875813804
Epoch: 3/5000, Train Loss: 36.432071685791016, Valid Loss: 37.100232442220054
Epoch: 4/5000, Train Loss: 35.90078700672496, Valid Loss: 36.76402918497721
Epoch: 5/5000, Train Loss: 35.76881478049538, Valid Loss: 36.70658747355143
Epoch: 6/5000, Train Loss: 35.52538576993075, Valid Loss: 36.704577128092446
Epoch: 7/5000, Train Loss: 35.651641845703125, Valid Loss: 36.701958974202476
Epoch: 8/5000, Train Loss: 35.69810901988637, Valid Loss: 36.70127487182617
Epoch: 9/5000, Train Loss: 35.67913471568715, Valid Loss: 36.69807688395182
Epoch: 10/5000, Train Loss: 35.60965243252841, Valid Loss: 36.689982096354164
Epoch: 11/5000, Train Loss: 35.56246358698065, Valid Loss: 36.67052459716797
Epoch: 12/5000, Train Loss: 35.55796120383523, Valid Loss:

In [102]:
torch.save(model_MLM.state_dict(), '/content/drive/MyDrive/Colab Notebooks/AIDrug_Competition/models/Baseline_Model_MLM_1.pth')  # 모델 객체의 state_dict 저장
torch.save(model_HLM.state_dict(), '/content/drive/MyDrive/Colab Notebooks/AIDrug_Competition/models/Baseline_Model_HLM_1.pth')

In [77]:
model_MLM.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/AIDrug_Competition/models/Baseline_Model_MLM_1.pth'))
model_HLM.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/AIDrug_Competition/models/Baseline_Model_HLM_1.pth'))

<All keys matched successfully>

In [103]:
test_MLM = CustomDataset(df=test, target=None, transform=transform, is_test=True)
test_HLM = CustomDataset(df=test, target=None, transform=transform, is_test=True)

test_MLM_loader = DataLoader(dataset=test_MLM,
                             batch_size=CFG['BATCH_SIZE'],
                             shuffle=False)

test_HLM_loader = DataLoader(dataset=test_HLM,
                             batch_size=CFG['BATCH_SIZE'],
                             shuffle=False)

In [104]:
def inference(test_loader, model):
    model.eval()
    preds = []

    with torch.no_grad():
        for inputs in test_loader:
            output = model(inputs)
            preds.extend(output.cpu().numpy().flatten().tolist())

    return preds

In [105]:
predictions_MLM = inference(test_MLM_loader, model_MLM)
predictions_HLM = inference(test_HLM_loader, model_HLM)

In [106]:
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AIDrug_Competition/data/sample_submission.csv')

submission['MLM'] = predictions_MLM
submission['HLM'] = predictions_HLM
submission

,id,MLM,HLM
0,TEST_000,10.873163,60.766888
1,TEST_001,67.025696,74.797096
2,TEST_002,23.396788,58.639282
3,TEST_003,40.519432,58.452656
4,TEST_004,44.585949,65.924644
...,...,...,...
478,TEST_478,10.913170,47.529285
479,TEST_479,73.660805,79.352829
480,TEST_480,50.021492,77.191963
481,TEST_481,61.042534,77.106689


In [107]:
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/AIDrug_Competition/submissions/Baseline_Model_1_submission.csv', index=False)